<a href="https://colab.research.google.com/github/ElanElanElanElan/DiffusingColab/blob/InTesting/DiffusingColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diffusing colab by using python! the way god intended**
---



In [ ]:
#@title **Installing** and **importing** all neccesary and semi neccesary stuff
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
import time
from PIL import Image
import torch
from IPython.utils import io
import re
import cv2
from google.colab.patches import cv2_imshow
with io.capture_output() as captured:
    !pip install --upgrade gradio diffusers accelerate transformers xformers safetensors
    output_text = captured.stdout
    errors = re.findall(r'(?i).error.', output_text)
    for error in errors:
        print(error)
import gradio as gr
from diffusers import DiffusionPipeline, StableDiffusionPipeline, StableDiffusionXLPipeline, AutoencoderTiny, EulerDiscreteScheduler, DDPMScheduler, KDPM2DiscreteScheduler, DPMSolverMultistepScheduler

In [ ]:
#@title use drive(optional) and create the setup folders

USE_GDRIVE = False #@param {type: "boolean"}
if USE_GDRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
!mkdir Loras
!mkdir Outputs
!mkdir Plots
lora_folder = '/content/Loras'
output_folder = '/content/Outputs'
plot_folder = "/content/Plots"

In [ ]:
#@title select which model and optimization to use
Enter_model = "BaseSdxl" # @param ["BaseSdxl", "base-1.5", "openjourney(realistic1.5)" ]
if Enter_model == 'BaseSdxl':
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    Pipeline =  StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
elif Enter_model == "base-1.5":
    model_id = "runwayml/stable-diffusion-v1-5"
    Pipeline =  StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
elif Enter_model == "openjourney(realistic1.5)":
    model_id = "upro/openjourney"
    Pipeline =  StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
enter_optimisation = "optimised" #@param ["none",'optimised']
if enter_optimisation == "optimised" and Enter_model == 'BaseSdxl':
    Pipeline.vae = AutoencoderTiny.from_pretrained("madebyollin/taesdxl", torch_dtype=torch.float16)
    Pipeline.enable_attention_slicing()
    Pipeline.enable_vae_slicing()
    Pipeline.enable_vae_tiling()
elif enter_optimisation == "optimised":
    Pipeline.vae = AutoencoderTiny.from_pretrained("madebyollin/taesd", torch_dtype=torch.float16)
    Pipeline.enable_attention_slicing()
    Pipeline.enable_vae_slicing()
    Pipeline.enable_vae_tiling()
if enter_optimisation == "none":
    Pipeline.disable_attention_slicing()
    Pipeline.disable_vae_tiling()
    Pipeline.disable_vae_slicing()

In [ ]:
#@title run the cell below to define all the functions you will use (feel free to inspect)
def Loading_Lora(lora_name_string):
    lora_list = os.listdir(lora_folder)
    matching_files = [filename for filename in lora_list if lora_name_string in filename]

    if not matching_files:
        print(f"No matching files found for '{lora_name_string}' in {lora_folder}.")
        return None

    if len(matching_files) == 1:

        selected_file = matching_files[0]
        Pipeline.load_lora_weights(os.path.join(lora_folder, selected_file))
        return Pipeline
    else:
        print("more than one lora with matching names, be more specific in the naming")

def basic_generator(input_prompt: str, neg_prompt:str, number_of_pics: int, steps: int, width = 1024, height = 1024, Fixed_seed: bool = False, Use_Lora: bool = False, Lora_name: str = "", Lora_comparision: bool = False, scheduler="DPMSolverMultistep") -> list:
    number_of_pics = int(number_of_pics)
    steps = int(steps)
    width = int(width)
    height = int(height)
    output = []
    if len(Lora_name) == 0:
        Lora_name = None
    generator= None
    if scheduler=="EulerDiscrete":
      Pipeline.scheduler = EulerDiscreteScheduler.from_config(Pipeline.scheduler.config)
    elif scheduler=="DDPM":
      Pipeline.scheduler = DDPMScheduler.from_config(Pipeline.scheduler.config)
    elif scheduler=="KDPM2Discrete":
      Pipeline.scheduler = KDPM2DiscreteScheduler.from_config(Pipeline.scheduler.config)
    elif scheduler=="DPMSolverMultistep":
      Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
    Pipeline.to('cuda')
    if Use_Lora and Lora_name is None:
        raise ValueError("If 'Use_Lora' is True, 'Lora_name' must be provided.")
    elif not Use_Lora and Lora_name is not None:
        raise ValueError("If 'Use_Lora' is False, 'Lora_name' should not be provided.")
    if Use_Lora:
        Loading_Lora(Lora_name)
        for i in range(number_of_pics):
            with torch.no_grad():
                if Fixed_seed:
                    generator = torch.Generator(device = "cuda").manual_seed(0)
            output.append(Pipeline(prompt = input_prompt, num_inference_steps= steps, generator = generator, width = width, height = height).images)
        Pipeline.unload_lora_weights()
    elif Lora_comparision:
        output = []
        for lora in os.listdir(lora_folder):
            Loading_Lora(lora)
            with torch.no_grad():
                if Fixed_seed:
                    generator = torch.Generator(device = "cuda").manual_seed(0)
            output.append(Pipeline(prompt = input_prompt, num_inference_steps= steps, generator = generator, width = width, height = height).images)
            Pipeline.unload_lora_weights()
    else:
        for i in range(number_of_pics):
            with torch.no_grad():
                 if Fixed_seed:
                    generator = torch.Generator(device = "cuda").manual_seed(0)
            output.append(Pipeline(prompt = input_prompt, num_inference_steps= steps, generator = generator, width = width, height = height).images)
    image_paths = []
    for index, image in enumerate(output):
        file_name = f"image{index}.png"
        if file_name in os.listdir(output_folder):
            file_name = f"image{len(os.listdir(output_folder))}.png"
        image[0].save(os.path.join(output_folder, file_name), "PNG")
        image_paths.append(os.path.join(output_folder,file_name))

    return image_paths

def plot_images(image_paths:list = None):
    if image_paths is None:
        image_paths = [
            os.path.join(output_folder, filename)
            for filename in os.listdir(output_folder)
            if os.path.isfile(os.path.join(output_folder, filename))
        ]
    num_images = len(image_paths)
    num_rows = int(num_images ** 0.5)
    num_cols = (num_images + num_rows - 1) // num_rows
    total_width = max(img.width for img in map(Image.open, image_paths)) * num_cols
    total_height = max(img.height for img in map(Image.open, image_paths)) * num_rows
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(total_width/100 , total_height/100))
    for i, ax in enumerate(axes.ravel()):
        if i < len(image_paths):
            img = Image.open(image_paths[i])
            ax.imshow(img)
            ax.axis('off')
    plt.subplots_adjust(wspace=0.01, hspace=0.01)
    i = len(os.listdir(plot_folder))
    while True:
        plt_name = os.path.join(plot_folder, f"plot{i}.jpg")
        if plt_name in os.listdir(plot_folder):
            plt_name = os.path.join(plot_folder, f"plot{i+1}.jpg")
            i += 1
        else:
            break

    plt.savefig(plt_name, bbox_inches="tight")

def showcase(Image_paths : list):
    for filename in Image_paths:
        img = cv2.imread(filename)
        cv2_imshow(img)

In [ ]:
# @title Generate

prompt = "a woman in a bikini on a beach, colorful, happy environment" # @param {type:"string"}
neg_prompt = "blur, pixels, bad anatomy, bad hands, three hands, three legs, bad arms, missing legs, missing arms, poorly drawn face, bad face, fused face, cloned face, worst face, three crus, extra crus, fused crus, worst feet, three feet, fused feet, fused thigh, three thigh, fused thigh, extra thigh, worst thigh, missing fingers, extra fingers, ugly fingers, long fingers, horn, realistic photo, extra eyes, huge eyes, 2girl, amputation, disconnected limbs" # @param {type:"string"}
height = 1024 # @param {type:"integer"}
width = 1024 # @param {type:"integer"}
images = 1 # @param {type:"integer"}
steps = 20 # @param {type:"integer"}
Fixed_seed = True #@param {type: "boolean"}
clean_before_gen = False # @param {type:"boolean"}
scheduler = "DPMSolverMultistep" # @param ["EulerDiscrete","DDPM","KDPM2Discrete","DPMSolverMultistep"]
plot_generated_images = False #@param {type:'boolean'}
Use_Lora = False #@param {type:'boolean'}
Lora_name = '' # @param {type:"string"}
Lora_comparision = False #@param {type: "boolean"}
# @markdown comapares all loras in the lora folder and plots them (disregards number of pics)
if clean_before_gen:
  !find {output_folder} -type f -delete
  !find {plot_folder} -type f -delete
if plot_generated_images:
    plot_images(basic_generator(prompt,neg_prompt,images,steps,height = height,width= width, scheduler = scheduler, Fixed_seed = Fixed_seed, Use_Lora = Use_Lora, Lora_name = Lora_name))
else:
    showcase(basic_generator(prompt,neg_prompt,images,steps,height = height,width= width, scheduler = scheduler, Fixed_seed = Fixed_seed, Use_Lora = Use_Lora, Lora_name = Lora_name))

**Utility cells below**
---



In [ ]:
#@title run the cell to copy stuff from gdrive like loras, to the appropriate folders
user_input = ".safetensors" #@param {type:"string"}
google_drive_folder = "/content/drive/MyDrive/Ai downloadables (sfw) /Ai art/Ai art Downloadables" #@param {type:"string"}
directoryto = lora_folder # @param ["lora_folder"] {type:"raw"}




full_content = os.listdir(google_drive_folder)


for file in full_content:
    if user_input in file:
        source_path = os.path.join(google_drive_folder, file)
        shutil.copy(source_path, directoryto)


In [ ]:
#@title run the **`Gradio`** interface (not completed)
#Iface = gr.Interface(fn=user_inputs, inputs= ["text","number"], outputs= "image")
#Iface.launch(debug=True)